In [83]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import GridSearchCV, StratifiedKFold




In [ ]:

# Load the data
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")


In [ ]:
# Feature Engineering
# Extract Title from Name, replace rare titles with 'Rare'
train_data['Title'] = train_data['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
test_data['Title'] = test_data['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
rare_titles = ['Lady', 'Countess','Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona']
train_data['Title'] = train_data['Title'].replace(rare_titles, 'Rare')
test_data['Title'] = test_data['Title'].replace(rare_titles, 'Rare')

# Create 'FamilySize' feature and 'IsAlone' feature
train_data['FamilySize'] = train_data['SibSp'] + train_data['Parch'] + 1
test_data['FamilySize'] = test_data['SibSp'] + test_data['Parch'] + 1
train_data['IsAlone'] = 0
train_data.loc[train_data['FamilySize'] == 1, 'IsAlone'] = 1
test_data['IsAlone'] = 0
test_data.loc[test_data['FamilySize'] == 1, 'IsAlone'] = 1

# Fill NaN values in 'Age' column with the median age
train_data['Age'].fillna(train_data['Age'].median(), inplace=True)
test_data['Age'].fillna(test_data['Age'].median(), inplace=True)

# Bin 'Age' into categories
train_data['AgeBin'] = pd.cut(train_data['Age'].astype(int), 5, labels=['Child','Teenager','Adult','MiddleAged', 'Elderly'])
test_data['AgeBin'] = pd.cut(test_data['Age'].astype(int), 5, labels=['Child','Teenager','Adult','MiddleAged', 'Elderly'])


# Bin 'Age' into categories
train_data['AgeBin'] = pd.cut(train_data['Age'].astype(int), 5, labels=['Child','Teenager','Adult','MiddleAged', 'Elderly'])
test_data['AgeBin'] = pd.cut(test_data['Age'].astype(int), 5, labels=['Child','Teenager','Adult','MiddleAged', 'Elderly'])

# Bin 'Fare' into categories
train_data['FareBin'] = pd.qcut(train_data['Fare'], 4, labels=['Low', 'Medium', 'High', 'VeryHigh'])
test_data['FareBin'] = pd.qcut(test_data['Fare'], 4, labels=['Low', 'Medium', 'High', 'VeryHigh'])


In [ ]:
# Define preprocessing for numeric columns (normalize them so they're on the same scale)
numeric_features = ['FamilySize']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

# Define preprocessing for categorical features (encode them as one-hot vectors)
categorical_features = ['Embarked', 'Sex', 'Pclass', 'Title', 'IsAlone', 'AgeBin', 'FareBin']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])


In [ ]:

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Split the data
X = train_data.drop('Survived', axis=1)
y = train_data['Survived']

In [87]:
# Create a preprocessing and classification pipeline
rf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier())])

# Define the parameter grid
param_grid = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [None, 5, 10, 15],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4],
    'classifier__max_features': ['sqrt']  # Changed 'auto' to 'sqrt'
}


# Create the grid search object
grid_search = GridSearchCV(rf, param_grid, cv=StratifiedKFold(n_splits=5))

# Fit the model
grid_search.fit(X, y)

# Print the best parameters
print("Best parameters: ", grid_search.best_params_)

# Use the best model to make predictions
best_rf = grid_search.best_estimator_
predictions = best_rf.predict(test_data)

Best parameters:  {'classifier__max_depth': 5, 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 2, 'classifier__min_samples_split': 5, 'classifier__n_estimators': 200}


In [91]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = grid_search, X = X, y = y, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 81.49 %
Standard Deviation: 4.56 %


In [92]:
# Create a DataFrame with the passenger IDs and our prediction regarding whether they survived or not
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})

# Write the DataFrame to a CSV file
output.to_csv('submission6.csv', index=False)
